In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [29]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [30]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [31]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [35]:
df_fake["label"] = "1"
df_true["label"] = "0"
df = pd.concat([df_fake, df_true])
df = df.sample(frac = 1).reset_index(drop = True)

In [36]:
df.head()

,title,text,subject,date,label
0,Trump and Obama set campaign rancor aside with...,WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"November 10, 2016",0
1,Jake Tapper BLASTS Trump’s ‘Do A Lot More Wit...,Jake Tapper has just about had it with Donald ...,News,"March 3, 2017",1
2,"Trump welcomes stock market surge, consumer co...",WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"March 2, 2017",0
3,Kazakh leader tells cabinet: Make state firms ...,ASTANA (Reuters) - Kazakh President Nursultan ...,worldnews,"December 6, 2017",0
4,WOW! LEGAL HISPANIC AMERICAN IMMIGRANT Unloads...,,politics,"Oct 14, 2016",1


In [37]:
df.text = df.title+df.text
df.drop(columns = ["title", "subject", "date"], axis = 1, inplace = True)
df.head()

,text,label
0,Trump and Obama set campaign rancor aside with...,0
1,Jake Tapper BLASTS Trump’s ‘Do A Lot More Wit...,1
2,"Trump welcomes stock market surge, consumer co...",0
3,Kazakh leader tells cabinet: Make state firms ...,0
4,WOW! LEGAL HISPANIC AMERICAN IMMIGRANT Unloads...,1


In [38]:
X = df["text"]
y = df["label"]

In [43]:
stop_words = set(stopwords.words('english')) 
def LemmSentence(sentence):
    lemma_words = []
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(sentence) 
    for word in word_tokens: 
        if word not in stop_words: 
            new_word = re.sub('[^a-zA-Z]', '',word)
            new_word = new_word.lower()
            new_word = wordnet_lemmatizer.lemmatize(new_word)
            lemma_words.append(new_word)
    return " ".join(lemma_words)

X = [LemmSentence(i) for i in X]

In [56]:
X = pd.DataFrame(X)
y = pd.DataFrame(y)

In [61]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [62]:
vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(x_train.iloc[:,0])
tfidf_test = vectorizer.transform(x_test.iloc[:,0])

In [63]:
pac = PassiveAggressiveClassifier(random_state = 7,loss = 'squared_hinge',  max_iter = 50, C = 0.16)
pac.fit(tfidf_train, y_train.values.ravel())
y_pred = pac.predict(tfidf_test)
score = round((accuracy_score(y_test, y_pred)*100), 2)

print("accuracy=",score)

accuracy= 99.69
